In [3]:
import xml.etree.cElementTree as et
from lxml import etree
import pandas as pd
import os
import re

In [4]:
# All files
filePath='/home/ec2-user/SageMaker/data/corpus/regnews_aug2020/'
files=[]
for file in os.listdir(filePath):
    files.append(file)
print(len(files))
#print(files[0:5])

for file in files:
    if file.endswith('.xml'):
        pass
    else:
        print(file)

853284


In [5]:
# Function to print one XML example
def print_xml(file):
    tree = etree.parse(file)
    xml = etree.tostring(tree, encoding="unicode", pretty_print=True)
    print(xml)

In [6]:
print_xml(filePath+files[0])

<RECORD>
   <GOID>433947392</GOID>
   
   <Obj>
      <SourceRollupType>Newspapers</SourceRollupType>
      <ObjectTypes>
         <other ObjectTypeOrigin="Publisher">News</other>
         <mstar>News</mstar>
      </ObjectTypes>
      <ObjectRollupType>Articles - All Types</ObjectRollupType>
      <TitleAtt>
         <Title>Federal Officials Seek to Relax Rules for Dumping Mine Waste</Title>
      </TitleAtt>
      <SubTitleAtt>
         <SubTitle>[National Desk]</SubTitle>
      </SubTitleAtt>
      <NumericDate>2008-10-19</NumericDate>
      <StartDate>2008-10-19</StartDate>
      <EndDate>2008-10-19</EndDate>
      <AlphaDate>Oct 19, 2008</AlphaDate>
      <Language>       
         <RawLang>English</RawLang>
     </Language>
      <Language IsPrimary="true">       
         <ISO>         
            <ISOCode>ENG</ISOCode>
            <ISOExpansion ISOCode="ENG">English</ISOExpansion>
         </ISO>
     </Language>
      <Copyright>
         <CopyrightData>Copyright New York Tim

In [5]:
# Function to remove html tags from a string
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# Function to remove multiple spaces
def remove_spaces(text):
    text=re.sub(' +',' ',text).strip()
    return text

In [22]:
# Function to parse XML
def import_xml(ID, file):
    xmlp = et.XMLParser(encoding="UTF-8")
    parsed_xml = et.parse(file,parser=xmlp)
    root = parsed_xml.getroot()
    for child in root.findall('Obj'):
        lang=child.find('Language').find('RawLang').text
    if lang=='English':
        for child in root.findall('Obj'):
            type=child.find('ObjectTypes').find('mstar').text
            title=child.find('TitleAtt').find('Title').text
            try:
                startdate=child.find('StartDate').text
                enddate=child.find('EndDate').text
            except:
                startdate=child.find('NumericDate').text
                enddate=child.find('NumericDate').text

        if root.find('TextInfo')!=None:
            for node in root.iter('Text'):
                text=node.text
                text=remove_spaces(remove_html_tags(text))
                wordcount=node.get('WordCount')
        else:
            text=''
            wordcount=0

        for child in root.findall('DFS'):
            pubtitle=child.find('PubFrosting').find('Title').text
            sourcetype=child.find('PubFrosting').find('SourceType').text
    
        new_dic={'ID':ID,'Title':title,'Type':type,'StartDate':startdate,'EndDate':enddate,'Text':text,
            'TextWordCount':wordcount,'PubTitle':pubtitle, 'SourceType':sourcetype}
        df_xml=pd.DataFrame([new_dic])
    return df_xml

In [ ]:
# Parse XML files
df=pd.DataFrame()
not_parsed=[]
for filename in files:
    if filename.endswith('.xml'):
        ID=filename.split('.xml')[0]
        file=filePath+filename
        try:
            new=import_xml(ID, file)
            df=df.append(new).reset_index(drop=True)
        except:
            not_parsed.append(filename)
            print('Could not parse:',filename)

In [15]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853283 entries, 0 to 853282
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ID             853283 non-null  object
 1   Title          853283 non-null  object
 2   Type           853283 non-null  object
 3   StartDate      853283 non-null  object
 4   EndDate        853283 non-null  object
 5   Text           853283 non-null  object
 6   TextWordCount  853283 non-null  object
 7   PubTitle       853283 non-null  object
 8   SourceType     853283 non-null  object
dtypes: object(9)
memory usage: 58.6+ MB
None


In [16]:
print(df.head())

           ID                                              Title  Type  \
0   433947392  Federal Officials Seek to Relax Rules for Dump...  News   
1   292714709  Connecticut Seacoast Village Says `Enough' to ...  News   
2  1828076074               Toyota, Suzuki agree to explore deal  News   
3   421725277   Los Angeles; Limits on Solicitors at LAX Stalled  News   
4  2049477484                                      NEWS BRIEFING  News   

    StartDate     EndDate                                               Text  \
0  2008-10-19  2008-10-19  The Interior Department has advanced a proposa...   
1  1988-01-31  1988-01-31  This picturesque river town has adopted rules ...   
2  2016-10-12  2016-10-12  Japanese automakers Toyota and Suzuki said Wed...   
3  2002-05-22  2002-05-22  Citing a need for additional study, the city's...   
4  2018-06-04  2018-06-04  Flight 93 memorial tower set to open on 17th a...   

  TextWordCount                               PubTitle  SourceType  
0    

In [18]:
df.to_pickle('/home/ec2-user/SageMaker/New Uncertainty/parsed_xml_new.pkl')

In [ ]:
import pickle
with open('/home/ec2-user/SageMaker/New Uncertainty/not_parsed', 'wb') as fp:
    pickle.dump(not_parsed, fp)

In [20]:
# Not parsed XML files
print(len(not_parsed))
print(not_parsed)

1
['2305084552.xml']


In [19]:
print_xml(filePath+not_parsed[0])

<RECORD>
   <GOID>2305084552</GOID>
   
   <Obj>
      <SourceRollupType>Newspapers</SourceRollupType>
      <ObjectTypes>
         <other ObjectTypeOrigin="EXT">News</other>
         <mstar>News</mstar>
      </ObjectTypes>
      <ObjectRollupType>Articles - All Types</ObjectRollupType>
      <TitleAtt HTMLContent="true">
         <Title>Georgia shocked by South Carolina in College Football Playoff shakeup</Title>
      </TitleAtt>
      <NumericDate>2019-10-12</NumericDate>
      <AlphaDate>Oct 12, 2019</AlphaDate>
      <Language>       
         <RawLang>English</RawLang>
     </Language>
      <Language IsPrimary="true">       
         <ISO>         
            <ISOCode>ENG</ISOCode>
            <ISOExpansion ISOCode="ENG">English</ISOExpansion>
         </ISO>
     </Language>
      <Copyright>
         <CopyrightData>Copyright USA Today, a division of Gannett Satellite Information Network, Inc. Oct 2019</CopyrightData>
      </Copyright>
      <PrintLocation>       
         <

In [23]:
# Parse XML files in not_parsed
for filename in not_parsed:
    if filename.endswith('.xml'):
        ID=filename.split('.xml')[0]
        file=filePath+filename
        new=import_xml(ID, file)
        df=df.append(new).reset_index(drop=True)

In [24]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 853284 entries, 0 to 853283
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ID             853284 non-null  object
 1   Title          853284 non-null  object
 2   Type           853284 non-null  object
 3   StartDate      853284 non-null  object
 4   EndDate        853284 non-null  object
 5   Text           853284 non-null  object
 6   TextWordCount  853284 non-null  object
 7   PubTitle       853284 non-null  object
 8   SourceType     853284 non-null  object
dtypes: object(9)
memory usage: 58.6+ MB
None


In [27]:
print(df['Type'].value_counts())

News                      848427
Feature                     4060
Undefined                    668
General Information           32
Commentary                    31
Article                       30
Statistics/Data Report        10
Obituary                       8
Review                         7
Correction/Retraction          5
Recipe                         3
Interview                      2
Editorial                      1
Name: Type, dtype: int64


In [28]:
print(df['SourceType'].value_counts())

Newspapers    853284
Name: SourceType, dtype: int64


In [25]:
df.to_pickle('/home/ec2-user/SageMaker/New Uncertainty/parsed_xml.pkl')